In [1]:
!pip install tensorflow

# **CNN** MODEL

In [2]:
import pandas as pd
import numpy as np
import os
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image


csv_file_path = "HAM10000_metadata.csv"
df = pd.read_csv(csv_file_path)

In [3]:
# t_image_dir = ["/content/drive/MyDrive/Cancer/SkinCancer/SkinCancer/"+ i for i in t_image + ".jpg"]
# /content/drive/MyDrive/Cancer/SkinCancer/SkinCancer/ISIC_0027850.jpg

In [7]:
def load_and_preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.resize((64, 64))#224 ,224
    image = np.array(image)
    return image

In [5]:
# random.seed(42)
# selected_data = df.sample(n=5000, random_state=42)
# selected_data = df[:5000]

In [8]:
images = np.array([load_and_preprocess_image("C:\\Users\\91832\\OneDrive\\Documents\\Project 7 Sem\\Skin Cancer\\Skin Cancer/" + img_id + ".jpg")
                   for img_id in df['image_id']])

In [9]:
labels = df['dx']

In [10]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)
encoded_labels = np.eye(num_classes)[encoded_labels]

In [11]:
images.shape

(10015, 64, 64, 3)

In [12]:
encoded_labels[1]

array([0., 0., 1., 0., 0., 0., 0.])

In [13]:
train_images, val_images, train_labels, val_labels = train_test_split(images, encoded_labels, test_size=0.2)

In [14]:
def create_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [15]:
image_height, image_width, num_channels = images[0].shape

In [16]:
cnn_model = create_cnn_model()

In [17]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 6, 6, 128)         0

In [ ]:
num_epochs = 4
batch_size = 32
history = cnn_model.fit(train_images, train_labels,batch_size=batch_size, epochs=num_epochs, validation_data=(val_images, val_labels))

Epoch 1/4
251/251 [==============================] - 29s 113ms/step - loss: 2.9194 - accuracy: 0.6549 - val_loss: 0.9832 - val_accuracy: 0.6590
Epoch 2/4
251/251 [==============================] - 29s 114ms/step - loss: 0.8836 - accuracy: 0.6867 - val_loss: 1.0085 - val_accuracy: 0.6635
Epoch 3/4
251/251 [==============================] - 30s 118ms/step - loss: 0.8495 - accuracy: 0.6945 - val_loss: 0.8903 - val_accuracy: 0.6770
Epoch 4/4
199/251 [======================>.......] - ETA: 5s - loss: 0.8099 - accuracy: 0.7026

In [ ]:
# Save the Model

from tensorflow.keras.models import load_model

cnn_model.save(os.path.join('C:\\Users\\91832\\OneDrive\\Documents\\Project 7 Sem\\Skin Cancer\\','image_64x64_3.h5'))
# new_model = load_model('imageclassifier.h5')
# cnn_model.predict(np.expand_dims(resize/255, 0))


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

val_predictions = cnn_model.predict(val_images)

val_predicted_labels = np.argmax(val_predictions, axis=1)

val_true_labels = np.argmax(val_labels, axis=1)

conf_matrix = confusion_matrix(val_true_labels, val_predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)